# RNNによる自然言語処理
RNNを使って、文書の自動作成を行います。  
今回は、宮沢賢治の「銀河鉄道の夜」を学習データに使い、賢治風の文章を自動生成します。  
文章における文字の並びを時系列データと捉えて、次の文字を予測するようにRNNを訓練します。  
シンプルなRNN、LSTM、およびGRUの3つのRNNでそれぞれモデルを構築し、文章の生成結果を比較します。

## テキストデータの読み込み
Google ドライブからテキストデータを読み込みます。  
このノートブックと同じフォルダに、青空文庫の「銀河鉄道の夜」のテキストデータ"gingatetsudono_yoru.txt"がありますので、パスを指定して読み込みます。

In [5]:
from google.colab import drive

drive.mount('/content/drive/')
nov_dir = 'Udemy_activity/ai_master_course/Section_8/'  # このフォルダへのパス
nov_path = '/content/drive/My Drive/04_Google Colaboratory/191211_AI Perfect Master (Colab Notebooks)/ai_master_course/Section_8（RNN）/gingatetsudono_yoru.txt'

# ファイルを読み込む
with open(nov_path, 'r') as f:
  nov_text = f.read()
  print(nov_text[:1000])  # 最初の1000文字のみ表示

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
「ではみなさんは、そういうふうに川だと云《い》われたり、乳の流れたあとだと云われたりしていたこのぼんやりと白いものがほんとうは何かご承知ですか。」先生は、黒板に吊《つる》した大きな黒い星座の図の、上から下へ白くけぶった銀河帯のようなところを指《さ》しながら、みんなに問《とい》をかけました。
　カムパネルラが手をあげました。それから四五人手をあげました。ジョバンニも手をあげようとして、急いでそのままやめました。たしかにあれがみんな星だと、いつか雑誌で読んだのでしたが、このごろはジョバンニはまるで毎日教室でもねむく、本を読むひまも読む本もないので、なんだかどんなこともよくわからないという気持ちがするのでした。
　ところが先生は早くもそれを見附《みつ》けたのでした。
「ジョバンニさん。あなたはわかっているのでしょう。」
　ジョバンニは勢《いきおい》よく立ちあがりましたが、立って見るともうはっきりとそれを答えることができないのでした。ザネリが前の席からふりかえって、ジョバンニを見てくすっとわらいました。ジョバンニはもうどぎまぎしてまっ赤になってしまいました。先生がまた云いました。
「大きな望遠鏡で銀河をよっく調べると銀河は大体何でしょう。」
　やっぱり星だとジョバンニは思いましたがこんどもすぐに答えることができませんでした。
　先生はしばらく困ったようすでしたが、眼《め》をカムパネルラの方へ向けて、
「ではカムパネルラさん。」と名指しました。するとあんなに元気に手をあげたカムパネルラが、やはりもじもじ立ち上ったままやはり答えができませんでした。
　先生は意外なようにしばらくじっとカムパネルラを見ていましたが、急いで「では。よし。」と云いながら、自分で星図を指《さ》しました。
「このぼんやりと白い銀河を大きないい望遠鏡で見ますと、もうたくさんの小さな星に見えるのです。ジョバンニさんそうでしょう。」
　ジョバンニはまっ赤になってうなずきました。けれどもいつかジョバンニの眼のなかには涙《なみだ》がいっぱいになりました。そう

## 正規表現による前処理
正規表現を使って、ルビなどを除去します。

In [6]:
import re  # 正規表現に必要なライブラリ

text = re.sub("《[^》]+》", "", nov_text) # ルビの削除
text = re.sub("［[^］]+］", "", text) # 読みの注意の削除
text = re.sub("[｜ 　]", "", text) # | と全角半角スペースの削除
print("文字数", len(text))  # len() で文字列の文字数も取得可能

文字数 38753


## 各設定
RNNの各設定です。

In [7]:
n_rnn = 10  # 時系列の数
batch_size = 128
epochs = 60
n_mid = 128  # 中間層のニューロン数

## 文字のベクトル化
各文字をone-hot表現で表し、時系列の入力データおよび正解データを作成します。  
今回はRNNの最後の時刻の出力のみ利用するので、最後の出力に対応する正解のみ必要になります。

In [8]:
import numpy as np

# インデックスと文字で辞書を作成
chars = sorted(list(set(text)))  # setで文字の重複をなくし、各文字をリストに格納する
print("文字数（重複無し）", len(chars))
char_indices = {}  # 文字がキーでインデックスが値
for i, char in enumerate(chars):
    char_indices[char] = i
indices_char = {}  # インデックスがキーで文字が値
for i, char in enumerate(chars):
    indices_char[i] = char
 
# 時系列データと、それから予測すべき文字を取り出します
time_chars = []
next_chars = []
for i in range(0, len(text) - n_rnn):
    time_chars.append(text[i: i + n_rnn])
    next_chars.append(text[i + n_rnn])
 
# 入力と正解をone-hot表現で表します
x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=np.bool)
t = np.zeros((len(time_chars), len(chars)), dtype=np.bool)
for i, t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1  # 正解をone-hot表現で表す
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1  # 入力をone-hot表現で表す
        
print("xの形状", x.shape)
print("tの形状", t.shape)

文字数（重複無し） 1049
xの形状 (38743, 10, 1049)
tの形状 (38743, 1049)


## モデルの構築
SimpleRNN、LSTM、GRUの層を使ったモデルをそれぞれ構築します。

In [9]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU

# SimpleRNN
model_rnn = Sequential()
model_rnn.add(SimpleRNN(n_mid, input_shape=(n_rnn, len(chars))))
model_rnn.add(Dense(len(chars), activation="softmax"))
model_rnn.compile(loss='categorical_crossentropy', optimizer="adam")
print(model_rnn.summary())

# LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(n_mid, input_shape=(n_rnn, len(chars))))
model_lstm.add(Dense(len(chars), activation="softmax"))
model_lstm.compile(loss='categorical_crossentropy', optimizer="adam")
print(model_lstm.summary())

# GRU
model_gru = Sequential()
model_gru.add(GRU(n_mid, input_shape=(n_rnn, len(chars))))
model_gru.add(Dense(len(chars), activation="softmax"))
model_gru.compile(loss='categorical_crossentropy', optimizer="adam")
print(model_gru.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               150784    
_________________________________________________________________
dense (Dense)                (None, 1049)              135321    
Total params: 286,105
Trainable params: 286,105
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               603136    
_________________________________________________________________
dense_1 (Dense)              (None, 1049)              135321    
Total params: 738,457
Trainable params: 738,457
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequ

## 文書生成用の関数
各エポックの終了後、文章を生成するための関数を記述します。  
LambdaCallbackを使って、エポック終了時に実行される関数を設定します。

In [11]:
from keras.callbacks import LambdaCallback
 
def on_epoch_end(epoch, logs):
    print("エポック: ", epoch)

    beta = 5  # 確率分布を調整する定数
    prev_text = text[0:n_rnn]  # 入力に使う文字
    created_text = prev_text  # 生成されるテキスト
    
    print("シード: ", created_text)

    for i in range(400):
        # 入力をone-hot表現に
        x_pred = np.zeros((1, n_rnn, len(chars)))
        for j, char in enumerate(prev_text):
            x_pred[0, j, char_indices[char]] = 1
        
        # 予測を行い、次の文字を得る
        y = model.predict(x_pred)
        p_power = y[0] ** beta  # 確率分布の調整
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))        
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + next_char

    print(created_text)
    print()

# エポック終了後に実行される関数を設定
epock_end_callback= LambdaCallback(on_epoch_end=on_epoch_end)

## 学習
構築したモデルを使って、学習を行います。  
fit( )メソッドをではコールバックの設定をし、エポック終了後に関数が呼ばれるようにします。  
学習には時間がかかりますので、編集→ノートブックの設定のハードウェアアクセラレーターでGPUを選択しましょう。

In [ ]:
# シンプルなRNN
model = model_rnn
history_rnn = model_rnn.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])

Epoch 1/60
303/303 [==============================] - 23s 10ms/step - loss: 5.3109
エポック:  0
シード:  「ではみなさんは、そ
「ではみなさんは、そたたとたましたののの、したののいのののののいまのしのまのたののいのいた。うていた。いまのたののいのてのうまたたののかまのいうてた、、たたたらいのののう、のののたたのしたてっしたのまかのたたののうのしのののの、のうの。ののいのののしていのののった、まのたうの、ののしのの、のたのたうたまのたのいした。たのうのうの、ま、たのののいのいのうがのいののたたのののう、いのしのたいがしてのしらうていのまたたいのをいたのかののののううまた、うのうの、たはにのののていたのいたいのののいののいたいのっ。たのたいましたうしのいののしたまったたののいいいのかたいまいっうののしののたのののうのうののて、のかののてたたの、たののたのしのま、いていうたいにたな。ののののまって。のたいいのたたのたうののののうののの、、し。のしたのたいののしののま、ののののして。いまたたが、のた。たののい、うのののかい、のたいののいたうか

Epoch 2/60
303/303 [==============================] - 3s 11ms/step - loss: 4.6555
エポック:  1
シード:  「ではみなさんは、そ
「ではみなさんは、そのでした。
「あの、のはのにのした。
「のの、はの、ののにになてました。
「どは、のはのをのした。
「はした。
「ののはの、ののにのかのなのっていました。
「この、うにでのていていいした。
「あのはのなのなになのようになした。
「した。
「のは、ののののはのにのでのらのにのにのした。」
「した。」
「はのでの、をにっていました。
「のがうにないました。
「のの、にはのはのにのした。
「ののはののののにに、ました。
「
のののはのかにないてした。
「そのでした。
「ののうのでした。
「あの、のはのはは、いのした。」
「ョはンニは、ってのはに、っていた。」
「はは、いののはに、なました。
「ののはのはにないてした。
「ののはははっていたのでした。
「ののうのののははいました。
「あの、のなのはになっていました。
「あ

In [ ]:
# LSTM
model = model_lstm
history_lstm = model_lstm.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])

In [ ]:
# GRU
model = model_gru
history_gru = model_gru.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])

今回のケースでは、RNN < LSTM < GRUの順で文章が自然に見えます。  
SimpleRNNでは昔の文脈を利用するのが難しいのですが、GRUではある程度利用できているようです。  
興味のある方は、様々な条件をトライし、より自然な文章の生成にトライしてみましょう。  

## 学習の推移
誤差の推移を確認します。

In [ ]:
import matplotlib.pyplot as plt

loss_rnn = history_rnn.history['loss']
loss_lstm = history_lstm.history['loss']
loss_gru = history_gru.history['loss']

plt.plot(np.arange(len(loss_rnn)), loss_rnn, label="RNN")
plt.plot(np.arange(len(loss_lstm)), loss_lstm, label="LSTM")
plt.plot(np.arange(len(loss_gru)), loss_gru, label="GRU")
plt.legend()
plt.show()

誤差はまだ収束していないので、さらにエポック数を重ねることにより結果は改善しそうです。  
今回は文章の生成を行いましたが、同様にしてRNNを市場予測や自動作曲などに応用することも可能です。

## さらに自然な文章の生成のために
さらに自然な文章生成が可能なモデルを作るために、例えば以下のようなアプローチが有効かもしれません。

* **入力を単語ベクトルにする**  
入力をone-hot表現ではなくword2vecなどの技術により作る単語ベクトルにします。  
これにより、入力の次元数が抑えられるだけではなく、単語同士の関係性がモデルの訓練前にすでに存在することになります。    
word2vecについては、Udemyコース「自然言語処理とチャットボット: AIによる文章生成と会話エンジン開発」で詳しく解説しています。

* **コーパスをさらに大きくする**  
一般的に、コーパスが大きいほどモデルの汎用性は高まります。  
しかしながら、学習かかる時間が長くなるのが問題です。  

* **最新のアルゴリズムを採用する**  
自然言語処理の分野では日々新しい技術が生まれ、論文などで発表されています。  
興味のある方は、そのような技術をモデルに取り入れてみましょう。